처리 단계 처 리 내 용 요 약
데이터 준비 BBC 데이터셋을 불러와서 기사 내용과 레이블을 추출.
데이터 전처리 불용어를 제거 후 기사 내용을 토큰화하고, 단어를 숫자로 인코딩하여 벡터로 변환.
기사 내용의 길이를 맞추기 위해 패딩을 추가.
레이블을 원-핫 인코딩하여 벡터로 변환.
데이터 분할 전체 데이터를 학습 데이터와 테스트 데이터로로.
모델 구성 RNN(Recurrent Neural Network) 모델을 구성
임베딩 레이어를 추가하여 단어 벡터를 입력으로 받는다.
RNN 층을 추가하여 시퀀스 데이터를 처리
출력 층을 추가하여 분류 결과를 예측
모델 컴파일 손실 함수, 최적화 알고리즘, 평가 지표를 설정.
모델 학습 학습 데이터를 사용하여 모델을 학습
모델 평가 테스트 데이터를 사용하여 모델의 성능을 평가.
정확도, F1 스코어 등을 계산하여 분류 성능을 확인.
모델 예측 새로운 기사를 입력으로 받아 모델을 사용하여 분류 결과를 예측

NLTK(Natural Language Toolkit)자연어 처리를 위한 파이썬 라이브러리
영어 불용어(stopwords)
자연어 처리 작업에서 의미가 없거나 분석에 영향이 크지 않는 일반적인 단어들
텍스트 데이터를 전처리 할 때 사용.
주로 문법적인 역할을 하는 단어들, 텍스트 분석에 자주 등장하지만 중요한 의미를
전달하지 않는 경우가 많다.
불용어를 제거함으로써 모델의 성능을 향상시킬 수 있다.
예) "a", "an", "the", "is", "of", "and" 등 문장의 문법적인 구조를 형성하는 데 도움을
주지만, 분석에는 큰 영향을 미치지 않는다.
nltk.corpus.stopwords 모듈을 통해 영어 불용어를 사용할 수 있다. 

In [2]:
import csv
import numpy as np
import nltk
from time import time

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM,Embedding

In [75]:
MY_VOCAB = 5000 # 학습용으로 사용할 단어의 전체 갯수
MY_EMBED = 64 # 벡터화할 임베딩 규모(차원)
MY_HIDDEN = 100 # RNN 셀의 규모(숨겨진 레이어층)
MY_LEN = 200 # 업로드할 기사의 전체 단어의 숫자

MY_SPLIT = 0.8 # 학습용 데이터 비율
MY_SAMPLE = 123 # 샘플로 사용할 기사 번호
MY_EPOCH = 100 # 반복 학습수
original = []
processed = []
labels = []

In [5]:
# NLTK(Natural Language Toolkit)에서 영어의 불용어(stop words)를 다운로드
nltk.download('stopwords') # 기사 분류에 의미없는 영어 단어 다운로드
MY_STOP = set(nltk.corpus.stopwords.words('english'))

# corpus : 텍스트 데이터를 분석/처리 위한 여러 문서나 말뭉치들

print('영어 제외어',MY_STOP)
print('제외어 갯수', len(MY_STOP))
print(type(MY_STOP))
print('the' in MY_STOP)

영어 제외어 {"wouldn't", 'again', "doesn't", "they've", 'any', 'about', 'mustn', "hasn't", 'my', 'during', "he's", 'more', 'aren', 'most', 'some', 'after', 'them', 'who', "shan't", 'so', "he'll", 'that', 'at', 'theirs', 'were', 't', 'all', 'yourselves', 'will', "hadn't", 'or', 'whom', "i'd", 'what', "she'd", 'ourselves', 'where', "wasn't", 'mightn', 'yours', 'as', "he'd", "needn't", 'between', "it'd", 'how', "that'll", "don't", 'if', 'was', 'above', 'here', 'those', 'down', 'up', 'couldn', 'doesn', 'in', "isn't", 'not', "it'll", 'wasn', "aren't", 'further', 'own', 'the', 'hadn', "i'm", 'do', 'of', 'their', 're', 'having', "we've", 'out', "we're", 'you', "i've", 'it', 'while', 'because', 'himself', "mightn't", 'be', "we'd", 'only', 'than', 'her', 'very', 'from', 'wouldn', "she's", "you'd", 'am', 'll', 'y', 'with', 'he', 'this', "you've", 'nor', 'being', 'on', 'few', "won't", 'until', "she'll", "they'd", 'we', 'ain', "couldn't", 'is', 'these', 'and', 'has', 'there', 'weren', "should've", 'sha

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\main\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [14]:
import os
print("현재 작업 디렉토리:", os.getcwd())


현재 작업 디렉토리: c:\VisualStudio-WorkSpace\conetDeepLearning\20250604


In [19]:
import csv

# 파일 경로 설정 (Windows에서는 이스케이프 처리 필요)
path = r'.\bbc-text.csv'


# 리스트 초기화
labels = []
original = []
processed = []

# CSV 파일 열기
with open(path, 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    header = next(reader)  # 헤더 읽기
    print("헤더:", header)

    # 내용 확인 (옵션)
    for row in reader:
        # print(row)
        labels.append(row[0])      # 예: 카테고리이름을 리스트에 추가
        original.append(row[1])    # 예: 뉴스 본문 내용을 리스트에 추가
        news = row[1] # 각행의 두 번째 열값을 news 변수에 할당

        # print('before:',news)#제외어 처리 전 문장
        for word in MY_STOP:
            mask = ' ' + word+ ' '
            news = news.replace(mask,' ')
        # print('after :', news) # 제외어 처리 후 문장
        processed.append(news) # 처리된 결과인 news를 processed 리스트에 추가
    print('제외어 처리 전체 기사 수: ',len(processed))
    print('레이블 데이터 수:', len(labels))
    print('Original 데이터 수:', len(original))
    print('News 데이터 수:',len(news))

헤더: ['category', 'text']
제외어 처리 전체 기사 수:  2225
레이블 데이터 수: 2225
Original 데이터 수: 2225
News 데이터 수: 785


In [ ]:


## ✅ 코드 내용 정리

```python
for row in reader:
    labels.append(row[0])       # 각 행의 카테고리 이름을 labels 리스트에 추가
    original.append(row[1])     # 각 행의 기사 내용을 original 리스트에 추가
    news = row[1]               # 각 행의 두 번째 열 값을 news 변수에 할당
```

👉 **→ `labels`, `original` 리스트에 읽은 기사를 한 줄씩 추가 후 `news`의 내용은 별도 저장**

---

```python
for word in MY_STOP:
    mask = ' ' + word + ' '
    news = news.replace(mask, ' ')
    processed.append(news)
```

👉 **→ `news`의 내용에서 불용어를 찾아서 양쪽 끝에 공백을 넣어 `mask`로 담은 후,
해당 `mask`를 공백으로 대체해 `news`에 저장하고,
불용어가 사라진 `news`의 내용을 `processed`에 저장해 놓는다.**

---

## ✅ 보충 설명

> `"and"`라는 불용어 제거 시,
> `"sand"`라는 단어까지 `"s "`로 바뀌어버리는 문제를 해결한다.

* 즉, `mask = ' ' + word + ' '` 로 공백을 넣은 이유는,
  `" and "`만 제거하고 `"sand"`는 건드리지 않기 위함이다.

---

## ✅ 핵심 요약

| 리스트         | 역할                        |
| ----------- | ------------------------- |
| `labels`    | 뉴스의 카테고리(예: 스포츠, 정치 등) 저장 |
| `original`  | 뉴스 본문 원문 저장               |
| `processed` | 불용어 제거된 뉴스 본문 저장          |

| 전략                        | 이유                          |
| ------------------------- | --------------------------- |
| `' ' + word + ' '` 사용     | 정확한 단어 매칭만 제거 (단어 일부 제거 방지) |
| `news.replace(mask, ' ')` | 공백으로 치환하여 단어 위치 유지          |

---


In [20]:
print('샘플 기사 원문 :',original[MY_SAMPLE])
print('샘플 기사 분류 :',labels[MY_SAMPLE])
print('샘플 기사 기사 데이터:',type(original[MY_SAMPLE]))
print('샘플 기사의 젠체 단어수:',str(len(original[MY_SAMPLE])).split())

print('\n제외어를 삭제한 샘플 기사:',processed[MY_SAMPLE])
print('제외어 삭제 샘플의 단어수:',str(len(processed[MY_SAMPLE])).split())
# 정수에 split() 함수를 적용할 수 없기 때문에
# 문자열을 분할하기 위해 split()함수를 문자열에 적용해아

샘플 기사 원문 : screensaver tackles spam websites net users are getting the chance to fight back against spam websites  internet portal lycos has made a screensaver that endlessly requests data from sites that sell the goods and services mentioned in spam e-mail. lycos hopes it will make the monthly bandwidth bills of spammers soar by keeping their servers running flat out. the net firm estimates that if enough people sign up and download the tool  spammers could end up paying to send out terabytes of data.   we ve never really solved the big problem of spam which is that its so damn cheap and easy to do   said malte pollmann  spokesman for lycos europe.  in the past we have built up the spam filtering systems for our users   he said   but now we are going to go one step further.    we ve found a way to make it much higher cost for spammers by putting a load on their servers.  by getting thousands of people to download and use the screensaver  lycos hopes to get spamming websites constantly

In [25]:
# 텍스트 데이터를 작은 단위로 분할하는 처리(객체생성>토큰화)
A_token = Tokenizer(num_words= MY_VOCAB,
oov_token='!') # out of vocaburary > "!"
A_token.fit_on_texts(processed)

print('총 기사 수:',A_token.document_count)
print('총 단어 수:',len(A_token.word_counts))
print('각 단어의 사용 횟수:',A_token.word_counts)
print('단어를 정수로:',A_token.word_index)

총 기사 수: 2225
총 단어 수: 29698
각 단어의 사용 횟수: OrderedDict([('tv', 525), ('future', 373), ('hands', 91), ('viewers', 87), ('home', 663), ('theatre', 94), ('systems', 134), ('plasma', 14), ('high', 559), ('definition', 99), ('tvs', 19), ('digital', 415), ('video', 363), ('recorders', 20), ('moving', 79), ('living', 81), ('room', 66), ('way', 740), ('people', 2045), ('watch', 111), ('radically', 10), ('different', 258), ('five', 536), ('years', 1003), ('time', 1322), ('according', 422), ('expert', 29), ('panel', 80), ('gathered', 38), ('annual', 169), ('consumer', 218), ('electronics', 83), ('show', 603), ('las', 31), ('vegas', 30), ('discuss', 59), ('new', 1978), ('technologies', 116), ('impact', 132), ('one', 1892), ('favourite', 110), ('pastimes', 1), ('us', 1972), ('leading', 151), ('trend', 67), ('programmes', 101), ('content', 219), ('delivered', 50), ('via', 129), ('networks', 188), ('cable', 71), ('satellite', 47), ('telecoms', 63), ('companies', 428), ('broadband', 257), ('service', 45



## ✅ 코드 내용

```python
A_token = Tokenizer(num_words=MY_VOCAB, oov_token='!')
```

### 🔹 인자 설명

* **`num_words`**:
  토큰화할 단어의 최대 개수를 지정.
  → 가장 빈도가 높은 상위 `num_words`개의 단어만 토큰화에 사용.

* **`oov_token='!'`**:
  사전에 없는 단어(OOV: Out-Of-Vocabulary)를 나타내는 토큰을 `'!'`로 지정.
  → 학습하지 않은 단어가 입력되면 `'!'`로 대체됨.

---

```python
A_token.fit_on_texts(processed)
```

### 🔹 동작 설명

* **불용어가 제거된 텍스트 리스트 `processed`** 를 입력으로 받아
* 각 단어에 고유한 **정수 인덱스**를 부여하며 내부 단어 사전을 구축함.

---

## ✅ 핵심 요약

| 용어               | 설명                               |
| ---------------- | -------------------------------- |
| `Tokenizer`      | 텍스트를 숫자 인덱스로 변환해주는 Keras 도구      |
| `fit_on_texts()` | 전체 문서로부터 단어의 빈도수를 학습해 정수 인덱스를 매핑 |
| `oov_token`      | 훈련 시 사전에 없는 단어를 처리할 수 있도록 설정     |
| 결과               | 각 단어에 대해 정수 인덱스가 부여된 단어 사전 생성됨   |

---


In [27]:
# 입력된 텍스트 데이터를 정수 시퀀스로 변환
A_tokenized = A_token.texts_to_sequences(processed)

print('토큰 처리된 데이터 type:',type(A_tokenized))
print('토큰 처리된 데이터 type:',len(A_tokenized))
print('토큰 처리된 데이터 type:',A_tokenized[MY_SAMPLE])

토큰 처리된 데이터 type: <class 'list'>
토큰 처리된 데이터 type: 2225
토큰 처리된 데이터 type: [3171, 1, 816, 877, 115, 136, 382, 347, 716, 28, 816, 877, 228, 1, 3172, 27, 3171, 1, 4868, 203, 569, 733, 1771, 126, 4025, 816, 260, 395, 3172, 700, 21, 1648, 3629, 2849, 2607, 1, 2325, 2551, 453, 2919, 570, 115, 63, 2291, 381, 7, 1160, 780, 1860, 2607, 11, 92, 1571, 1051, 1, 203, 281, 154, 1, 138, 364, 816, 1, 2225, 847, 2, 1, 1, 178, 3172, 139, 255, 1109, 816, 1, 726, 136, 2, 52, 60, 10, 818, 3921, 195, 41, 21, 56, 494, 245, 2607, 1362, 1, 2551, 382, 1021, 7, 780, 70, 3171, 3172, 700, 23, 1, 877, 3993, 453, 343, 322, 1394, 3, 1, 2, 3427, 582, 816, 877, 297, 1, 56, 203, 2296, 2404, 2, 3171, 2709, 1069, 660, 812, 1287, 3883, 1539, 1, 466, 224, 503, 1539, 1, 31, 96, 1, 681, 111, 2, 10, 1899, 912, 2, 381, 7, 1160, 1, 877, 11, 722, 256, 1, 1287, 224, 503, 111, 3172, 79, 70, 260, 395, 716, 28, 2, 3, 1, 4, 1604, 10, 823, 455, 158, 823, 455, 2, 569, 2179, 4025, 816, 260, 395, 891, 733, 1771, 126, 220, 3677, 569, 316, 86,

불용어 처리 데이터를 입력으로 받아서 각각 정수 시퀀스로 변환.
이전 단계에서 생성한 Tokenizer 객체의 단어 사전을 활용하여 각 단어를 정수로 매핑
텍스트 데이터를 수치형 데이터로 변환하면, 기사 데이터를 신경망 모델에 입력으로
사용할 수 있다.

In [29]:
longest = max([len(x) for x in A_tokenized])
print('제일 긴 기사 단어 수 :',longest)
shortest = min([len(x) for x in A_tokenized])
print('제일 짧은 기사 단어 수 :',shortest)

제일 긴 기사 단어 수 : 2279
제일 짧은 기사 단어 수 : 50


In [31]:
A_tokenized = pad_sequences(A_tokenized,
maxlen = MY_LEN,
padding='pre',
truncating='pre')
print('샘플 기사 길이 처리본\n',A_tokenized[MY_SAMPLE])
print('토큰 처리된 데이터 수;',len(A_tokenized))

샘플 기사 길이 처리본
 [   2 3427  582  816  877  297    1   56  203 2296 2404    2 3171 2709
 1069  660  812 1287 3883 1539    1  466  224  503 1539    1   31   96
    1  681  111    2   10 1899  912    2  381    7 1160    1  877   11
  722  256    1 1287  224  503  111 3172   79   70  260  395  716   28
    2    3    1    4 1604   10  823  455  158  823  455    2  569 2179
 4025  816  260  395  891  733 1771  126  220 3677  569  316   86 1051
  816  260  395 3677   23    1 1453  681  111  415  569 3171  760  367
  189   14    1 3883 1595    1 1376  347 3754   27 3172  200    7  660
  569  848  816 1771  569 3258   70 2068 4063 4054  416 3791   77 3629
 2849   11   21  816 1731    2    3    1  569    6 1430 4185  203 4868
  251  664   65  910  231  569    1 3261  136 2820  136 3172  780   70
 3171  297 3171  571  877    1 4868  203 3171  269  633  383  139   35
  233    1 2644  193 4459  610    3    1    2 3171 1966  264   32  231
   12   96  276  430 1539  379    1  816  111 2985  111  323   

maxlen 파라미터 (200)를 사용하여 패딩 후 시퀀스의 최대 길이를 지정하고,
padding 파라미터 (pre)와 truncating 파라미터 (pre)를 사용하여 패딩의 위치를 지정

In [32]:
longest = max([len(x) for x in A_tokenized])
print('제일 긴 기사 단어 수 :',longest)
shortest = min([len(x) for x in A_tokenized])
print('제일 짧은 기사 단어 수 :',shortest)

제일 긴 기사 단어 수 : 200
제일 짧은 기사 단어 수 : 200


In [35]:
C_token = Tokenizer()
C_token.fit_on_texts(labels)

print('총 기사 수:',C_token.document_count)
print('총 단어 수:',len(C_token.word_counts))
print('각 단어의 사용 횟수:',C_token.word_counts)
print('단어를 정수로:',C_token.word_index)

총 기사 수: 2225
총 단어 수: 5
각 단어의 사용 횟수: OrderedDict([('tech', 401), ('business', 510), ('sport', 511), ('entertainment', 386), ('politics', 417)])
단어를 정수로: {'sport': 1, 'business': 2, 'politics': 3, 'tech': 4, 'entertainment': 5}


In [39]:
C_tokenized = C_token.texts_to_sequences(labels)
C_tokenized = np.array(C_tokenized)

print(C_tokenized)

[[4]
 [2]
 [1]
 ...
 [5]
 [3]
 [1]]


1. 텍스트 데이터 준비:
text_data = ["This is an example sentence", "Another example sentence"]
2.Tokenizer 객체 생성:
tokenizer = Tokenizer()
3.텍스트 데이터 입력하여 단어 사전 구축 및 인덱스
tokenizer.fit_on_texts(text_data)
4.텍스트 데이터를 정수 시퀀스로 변환:
tokenizer.texts_to_sequences(text_data)
5.패딩 또는 자르기 수행:
pad_sequences(sequences, maxlen=max_length, padding='post')

In [53]:
from sklearn.model_selection import train_test_split

X_train, X_test,Y_train,Y_test = train_test_split(A_tokenized,C_tokenized,train_size = MY_SPLIT,shuffle=False)
Y_train = np.array(Y_train) - 1
Y_test = np.array(Y_test) - 1
print('학습용 입력 데이터 모양:',X_train.shape)
print('학습용 출력 데이터 모양:',Y_train.shape)
print('단어를 정수로:',C_token.word_index)

학습용 입력 데이터 모양: (1780, 200)
학습용 출력 데이터 모양: (1780, 1)
단어를 정수로: {'sport': 1, 'business': 2, 'politics': 3, 'tech': 4, 'entertainment': 5}


In [44]:
model = Sequential()
model.add(Embedding(input_dim=MY_VOCAB,output_dim=MY_EMBED))
model.add(Dropout(rate=0.5))
model.add(LSTM(units=MY_HIDDEN))
model.add(Dense(units=5,activation='softmax'))

# 데이터 한 번 넣어서 강제로 빌드
model.build(input_shape=(None, MY_LEN)) # None은 배치크기
print('RNN 요약')
print("=========")
model.summary()

RNN 요약


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 200, 64)        │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 200, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100)            │        66,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           505 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 386,505 (1.47 MB)

 Trainable params: 386,505 (1.47 MB)

 Non-trainable params: 0 (0.00 B)

In [89]:
model = Sequential()
model.add(Embedding(input_dim=MY_VOCAB,output_dim=MY_EMBED))
model.add(Dropout(rate=0.5))
model.add(LSTM(units=MY_HIDDEN))
model.add(Dense(units=5,activation='softmax'))

# 데이터 한 번 넣어서 강제로 빌드
model.build(input_shape=(None, MY_LEN)) # None은 배치크기
print('RNN 요약')
print("=========")
model.summary()

RNN 요약


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ (None, 200, 64)        │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 200, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ (None, 100)            │        66,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 5)              │           505 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 386,505 (1.47 MB)

 Trainable params: 386,505 (1.47 MB)

 Non-trainable params: 0 (0.00 B)

In [84]:
model = Sequential()
model.add(Embedding(input_dim=MY_VOCAB, output_dim=MY_EMBED))
model.add(Bidirectional(LSTM(units=MY_HIDDEN, dropout=0.3, recurrent_dropout=0.3)))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=5, activation='softmax'))
# 데이터 한 번 넣어서 강제로 빌드
model.build(input_shape=(None, MY_LEN)) # None은 배치크기
print('RNN 요약')
print("=========")
model.summary()

RNN 요약


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ (None, 200, 64)        │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 200)            │       132,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │        12,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 465,189 (1.77 MB)

 Trainable params: 465,189 (1.77 MB)

 Non-trainable params: 0 (0.00 B)

In [90]:
model.compile(optimizer='adam',
loss ='sparse_categorical_crossentropy',
metrics=['acc'])
print('학습 시작')

학습 시작


In [91]:
begin = time()
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',     # 모니터할 기준 (val_accuracy 도 가능)
    patience=20,             # 개선이 없더라도 기다릴 에포크 수
    restore_best_weights=True  # 가장 성능 좋았던 가중치로 복원
)

model.fit(X_train,Y_train,epochs=MY_EPOCH,verbose=1,batch_size=64,validation_split=0.2,callbacks=[early_stop])
end = time()
print('training time : {:.2f}초'.format(end-begin))

Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 108ms/step - acc: 0.2588 - loss: 1.6028 - val_acc: 0.3792 - val_loss: 1.5566
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - acc: 0.4113 - loss: 1.4695 - val_acc: 0.4045 - val_loss: 1.3466
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - acc: 0.4685 - loss: 1.1049 - val_acc: 0.5365 - val_loss: 0.9606
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - acc: 0.7397 - loss: 0.7610 - val_acc: 0.7584 - val_loss: 0.8444
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - acc: 0.8484 - loss: 0.6086 - val_acc: 0.8315 - val_loss: 0.6446
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - acc: 0.8456 - loss: 0.5662 - val_acc: 0.6461 - val_loss: 0.8698
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - acc: 0.8762 - loss: 0.4712 - val_acc: 0.8174 - val_loss: 0.6930
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - acc: 0.9605 - loss: 0.3115 - val_acc: 0.8876 - val_loss: 0.4001
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - a

In [54]:
import numpy as np
print(np.unique(X_train))
print(np.unique(Y_train))


[   0    1    2 ... 4997 4998 4999]
[0 1 2 3 4]


In [92]:
score = model.evaluate(X_test, Y_test,verbose=1)
print('최종 정확도: {:.2f}'.format(score[1]))

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - acc: 0.9585 - loss: 0.1768
최종 정확도: 0.96


In [61]:
pred = model.predict(X_test)
print('추측값\n',pred)
pred = pred.argmax(axis=1)
print('추측값(argmax 처리 후)\n',pred)
print('정답\n', Y_test.flatten())

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
추측값
 [[2.4528147e-04 4.6005131e-05 9.0065514e-05 9.3895760e-05 9.9952471e-01]
 [8.1739119e-05 8.1173515e-05 2.4854255e-04 9.9947256e-01 1.1600000e-04]
 [1.4789430e-04 7.8044087e-04 9.9835432e-01 5.7014608e-04 1.4727052e-04]
 ...
 [1.3354317e-03 4.1007495e-04 6.3403696e-04 1.0830734e-03 9.9653745e-01]
 [2.2577729e-02 1.8471897e-02 4.6591386e-02 1.5012217e-02 8.9734679e-01]
 [9.9897325e-01 4.4003405e-04 7.9373785e-05 1.5810432e-04 3.4910909e-04]]
추측값(argmax 처리 후)
 [4 3 2 0 0 3 1 4 4 1 2 2 1 4 0 4 4 1 0 2 3 1 0 1 3 2 2 0 0 2 1 1 1 1 4 1 2
 2 3 3 4 4 4 1 2 0 0 3 3 1 3 0 1 1 1 0 0 2 2 4 4 2 1 2 2 1 3 2 2 2 2 2 4 4
 3 2 0 2 0 3 4 0 0 4 3 4 3 0 3 0 0 4 4 1 4 4 2 1 0 3 3 2 1 0 1 4 0 2 4 0 0
 1 2 3 3 1 1 4 2 4 0 0 2 4 3 0 4 1 2 0 1 3 4 0 2 1 4 2 4 2 0 2 1 1 2 1 4 0
 1 4 1 0 0 4 3 2 3 2 2 0 0 0 1 3 4 1 0 1 0 1 3 1 1 1 1 0 0 0 1 1 4 1 1 1 0
 0 0 3 1 0 0 0 1 4 3 3 3 2 1 1 3 1 3 0 0 2 2 2 0 0 2 2 3 1 0 0 0 0 1 0 1 1
 2 1 0 2 0 3 3 0 3 1 4 1 0 1 3 3 2 4 1 4 1 

In [62]:
correct_predictions = 0
total_predictions = len(Y_test)
for i in range(total_predictions):
    if pred[i] == Y_test[i]:
        correct_predictions +=1
accuracy = correct_predictions/total_predictions
print('정확도', accuracy)

정확도 0.9415730337078652


In [64]:

news = ['Bayern Munich cruised to an emphatic 7-0 victory over VfL Bochum \
 5 to move to the top of the German Bundesliga.']

news = A_token.texts_to_sequences(news)
print(news)
print('총 단어 수 :',len(news[0]))

[[1, 4260, 1, 521, 1, 1, 163, 289, 434, 1308, 1, 1, 100, 521, 156, 521, 1202, 66, 1366, 1202, 684, 1]]
총 단어 수 : 22


In [66]:
news = pad_sequences(news, maxlen=MY_LEN,padding='pre',truncating='pre')
print('총단어수 : ', len(news[0]))

pred = model.predict(news)
pred = pred.argmax(axis=1)
print('RNN 추측값 : ',pred)

총단어수 :  200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
RNN 추측값 :  [3]
